In [1]:
import torch
import torch.nn as nn
import torchvision

# Load observations from the mnist dataset. The observations are divided into a training set and a test set
mnist_train = torchvision.datasets.FashionMNIST('./data/fashion', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.FashionMNIST('./data/fashion', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output

# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

C:\Users\Eirik\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [2]:

class ConvolutionalNeuralNetworkModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Model layers (includes initialized model variables):
        self.logits = nn.Sequential(
					nn.Conv2d(1, 32, kernel_size=5, padding=2), # 32@28x28
					nn.MaxPool2d(kernel_size=2), # 32@14x14
					nn.Conv2d(32, 64, kernel_size=5, padding=2), # 32@14x14
					nn.MaxPool2d(kernel_size=2), # 64@7x7 
					nn.Flatten(), 
					nn.ReLU(),
					nn.Linear(64 * 7 * 7, 1024), # Dense, 64@7x7
					nn.Flatten(),
					nn.ReLU(),
					nn.Linear(1 * 1024, 10)) # Dense, 1x1024

    # Predictor
    def f(self, x):
        return torch.softmax(self.logits(x), dim=1)

    # Cross Entropy loss
    def loss(self, x, y):
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

In [3]:
model = ConvolutionalNeuralNetworkModel()

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(20):
	for batch in range(len(x_train_batches)):
			loss = model.loss(x_train_batches[batch], y_train_batches[batch])  # Compute loss gradients
			loss.backward()  # Compute loss gradients
			optimizer.step()  # Perform optimization by adjusting W and b,
			optimizer.zero_grad()  # Clear gradients for next step

	print("accuracy = %s" % model.accuracy(x_test, y_test))

# With added convolution and max-pool layers, lr=1e-03, epochs=20: 

C:\Users\Eirik\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


accuracy = tensor(0.8655)
accuracy = tensor(0.8859)
accuracy = tensor(0.8961)
accuracy = tensor(0.9000)
accuracy = tensor(0.8999)
accuracy = tensor(0.9041)
accuracy = tensor(0.9030)
accuracy = tensor(0.9090)
accuracy = tensor(0.9128)
accuracy = tensor(0.9146)
accuracy = tensor(0.9089)
accuracy = tensor(0.9043)
accuracy = tensor(0.9056)
accuracy = tensor(0.9087)
accuracy = tensor(0.9143)
